# Test: make predictions with library ensemble!

In [10]:
import numpy as np
import pysindy as ps
import matplotlib.pyplot as plt
from scipy import interpolate
import os
from pathlib import Path

import warnings
from scipy.integrate.odepack import ODEintWarning
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=ODEintWarning)

# integration keywords for solve_ivp, typically needed for chaotic systems
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

# CWD
cwd = os.getcwd()

# Intepolate?
spline_flag = True

# Dataset definition
hare =  [20, 20, 52, 83, 64, 68, 83, 12, 36, 150, 110, 60, 7, 10, 70, 100, 92, 70, 10, 11, 137, 137, 18, 22, 52, 83, 18, 10, 9, 65],
lynx =  [32, 50, 12, 10, 13, 36, 15, 12, 6, 6, 65, 70, 40, 9, 20, 34, 45, 40, 15, 15, 60, 80, 26, 18, 37, 50, 35, 12, 12, 25],

dt = 2
t0 = 1845
t1 = 1904
t_train = np.arange(t0, t1, dt)
#t_train = np.reshape(t_train, (1,30))
X_train = np.concatenate((np.array(hare),np.array(lynx)), axis=0).transpose()
X0 = X_train[0]

def interpolation(t ,X, t_new):

    f_0 = interpolate.splrep(t, X[:,0])
    f_1 = interpolate.splrep(t, X[:,1])

    X_new = np.zeros((len(t_new),2))
    X_new[:,0] = interpolate.splev(t_new, f_0)
    X_new[:,1] = interpolate.splev(t_new, f_1)

    return X_new

if spline_flag:
    dt = 0.5
    t_train_new = np.arange(t0,t1,dt)
    X_train = interpolation(t_train, X_train, t_train_new)
    print(f"Interpolation used; dt = {dt}")

t_train = t_train_new

Interpolation used; dt = 0.5


# Initiate library syndy model
Ho notato che alla fine il file non salva mai il modello in se, ma semplicemente la lsita dei coffficienti (model.coef_list)

In [30]:
feature_names = ['hare','lynx']

# Instantiate and fit the SINDy model
library_ensemble_optimizer = ps.STLSQ()
model = ps.SINDy(feature_names=feature_names, 
                 optimizer=library_ensemble_optimizer)
model.fit(
    X_train,
    t=dt,
    library_ensemble=True,
    #ensemble=True,
    n_subset=90,
    quiet=True
    )
model.print()
coef_list= model.coef_list

t_test = np.arange(t0, t1, 0.001)  
plt.figure()
for w in coef_list:
    model.optimizer.coef_ = w
    sim = model.simulate(X0,t_test, integrator='odeint')
    plt.plot(t_test,sim[:,1])

(hare)' = 0.000
(lynx)' = 0.000


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 6 is different from 5)

<Figure size 432x288 with 0 Axes>